https://www.youtube.com/watch?v=M9Itm95JzL0

In [5]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        
        

In [6]:
import json

file_name = './sklearn-master/data/sentiment/books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].text

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [7]:
###PREP DATA
from sklearn.model_selection import train_test_split

train,test = train_test_split(reviews, test_size=1/3,random_state=42)

train_container = ReviewContainer(train)
test_container = ReviewContainer(test)


#cont.evenly_distribute()
#len(cont.reviews)

In [9]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

429
429


In [10]:
###Bag of Words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()



train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)


#vectorizer.fit(train_x)
#train_x_vectors = vectorizer.transform(train_x)

#print(train_x[0])
#print(train_x_vectors[0].toarray())



In [11]:
###Classification

#Linear SVM
from sklearn import svm

clf_svm = svm.SVC(kernel = 'linear')
clf_svm.fit(train_x_vectors,train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [12]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

In [13]:
#Naive Bayes
#from sklearn.naive_bayes import GaussianNB

#clf_nb = GaussianNB()
#clf_nb.fit(train_x_vectors, train_y)

#clf_nb.predict(test_x_vectors[0])

In [14]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(train_x_vectors, train_y)

clf_lr.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [29]:
###Evaluation
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_lr.score(test_x_vectors,test_y))

0.813953488372093
0.6372093023255814
0.8023255813953488


In [16]:
from sklearn.metrics import f1_score

print([Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
print(f1_score(test_y,clf_svm.predict(test_x_vectors),average = None, labels = [Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

['POSITIVE', 'NEUTRAL', 'NEGATIVE']
[0.81395349 0.         0.81395349]


/Users/jimmydoran/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [17]:
train_y.count(Sentiment.POSITIVE),train_y.count(Sentiment.NEUTRAL),train_y.count(Sentiment.NEGATIVE)

(429, 0, 429)

In [28]:
test_set = ['it was not good']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)



array(['NEGATIVE'], dtype='<U8')

In [156]:
###tuning model 
###grid search

from sklearn.model_selection import GridSearchCV

parameters = {'kernel':('linear','rbf'),'C':(1,4,8,16,32)}

tuned_svm = svm.SVC()

clf = GridSearchCV(tuned_svm, parameters, cv=5)
clf.fit(train_x_vectors, train_y)



GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [157]:
print(clf.score(test_x_vectors,test_y))

0.8162790697674419
